# JSON Conversions

This notebook demonstrates how to convert a Resource to JSON and vice-versa. 

In [1]:
from kgforge.core import KnowledgeGraphForge

In [2]:
import getpass

In [ ]:
token = getpass.getpass()

In [4]:
forge = KnowledgeGraphForge("../../configurations/demo-forge-nexus.yml", token=token)

In [5]:
from kgforge.core import KnowledgeGraphForge

# Imports

In [6]:
import json

In [7]:
from kgforge.core import Resource

In [8]:
def pp(x): print(json.dumps(x, indent=4))

## Resource to JSON

In [9]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [10]:
jane = Resource(type="Person", name="Jane Doe", address=address, email="(missing)")

In [11]:
john = Resource(type="Person", name="John Smith", email="john.smith@epfl.ch")

In [12]:
persons = [jane, john]

In [13]:
forge.register(persons)

<count> 2
<action> _register_many
<succeeded> True


In [14]:
association = Resource(type="Association", agent=persons)

In [15]:
forge.register(association)

<action> _register_one
<succeeded> True


In [16]:
pp(forge.as_json(association))

{
    "id": "https://bbp.epfl.ch/dke/kgforge_tests/ce4a3e48-7940-4782-956d-1da8643cb0d4",
    "type": "Association",
    "agent": [
        {
            "context": "file:../../../tests/data/shacl-model/context.json",
            "id": "https://bbp.epfl.ch/dke/kgforge_tests/5032135a-e4f6-4c9c-9e11-d9ba559db0e0",
            "type": "Person",
            "address": {
                "type": "PostalAddress",
                "country": "Switzerland",
                "locality": "Geneva"
            },
            "email": "(missing)",
            "name": "Jane Doe"
        },
        {
            "context": "file:../../../tests/data/shacl-model/context.json",
            "id": "https://bbp.epfl.ch/dke/kgforge_tests/5caa0abe-a1b8-4019-a559-1d904a0001fc",
            "type": "Person",
            "email": "john.smith@epfl.ch",
            "name": "John Smith"
        }
    ]
}


In [17]:
pp(forge.as_json(jane, store_metadata=True))

{
    "id": "https://bbp.epfl.ch/dke/kgforge_tests/5032135a-e4f6-4c9c-9e11-d9ba559db0e0",
    "type": "Person",
    "address": {
        "type": "PostalAddress",
        "country": "Switzerland",
        "locality": "Geneva"
    },
    "email": "(missing)",
    "name": "Jane Doe",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_createdAt": "2020-04-17T14:12:25.201554Z",
    "_createdBy": "https://staging.nexus.ocp.bbp.epfl.ch/v1/realms/bbp/users/agarcia",
    "_deprecated": false,
    "_incoming": "https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/dke/kgforge_tests/_/5032135a-e4f6-4c9c-9e11-d9ba559db0e0/incoming",
    "_outgoing": "https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/dke/kgforge_tests/_/5032135a-e4f6-4c9c-9e11-d9ba559db0e0/outgoing",
    "_project": "https://staging.nexus.ocp.bbp.epfl.ch/v1/projects/dke/kgforge_tests",
    "_rev": 1,
    "_self": "https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/dke/kgforge_tests/_/5032135a-

## JSON to Resource

In [18]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [19]:
jane = Resource(type="Person", name="Jane Doe", address=address, email="(missing)")

In [20]:
john = Resource(type="Person", name="John Smith", email="john.smith@epfl.ch")

In [21]:
persons = [jane, john]

In [22]:
association = Resource(type="Association", agent=[jane, john])

In [23]:
data = {
    "type": "Association",
    "agent": [
        {
            "type": "Person",
            "address": {
                "type": "PostalAddress",
                "country": "Switzerland",
                "locality": "Geneva",
            },
            "email": "(missing)",
            "name": "Jane Doe"
        },
        {
            "type": "Person",
            "email": "john.smith@epfl.ch",
            "name": "John Smith"
        }
    ]
}

In [24]:
resource = forge.from_json(data)

In [25]:
resource == association

True

In [26]:
print(forge.from_json(data, na="(missing)"))

{
    type: Association
    agent:
    [
        {
            type: Person
            address:
            {
                type: PostalAddress
                country: Switzerland
                locality: Geneva
            }
            name: Jane Doe
        }
        {
            type: Person
            email: john.smith@epfl.ch
            name: John Smith
        }
    ]
}
